## STD project

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

import random as rnd

## Train and model utils

In [ ]:
from utils.train import *

In [ ]:
from utils.models import *

## Mess the data

In [ ]:
rate_errors = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

## Test distilation

In [ ]:
teacher_model = LeNet(distilation=False)

batch_size, lr, num_epochs = 256, 0.9, 10
train_iter, val_iter, test_iter = load_data_fashion_mnist(batch_size, resize=None)
train_loss, train_acc, val_loss, val_acc, test_acc, test_loss = train(teacher_model, train_iter, val_iter, test_iter, num_epochs, lr, try_gpu()) #2 min

teacher_model.eval()
student_model = LeNet(distilation=True)
criterion = nn.CrossEntropyLoss()
distilation_loss = DistilationLoss(criterion, teacher_model, distillation_type='hard')
train_iter, val_iter, test_iter = load_data_fashion_mnist(batch_size, error_rate=0.5)
train_loss, train_acc, val_loss, val_acc, test_acc, test_loss = train(student_model, train_iter, val_iter, test_iter, num_epochs, lr, try_gpu(), loss=distilation_loss) #2 min

## Train

In [ ]:
import numpy as np

batch_size, lr, num_epochs = 256, 0.9, 10

train_loss_all = []
train_acc_all = []
val_loss_all = []
val_acc_all = []
test_loss_all = []
test_acc_all = []

for error_rate in rate_errors:
    train_iter, val_iter, test_iter = load_data_fashion_mnist(batch_size, error_rate=error_rate)
    train_loss, train_acc, val_loss, val_acc, test_acc, test_loss = train(net, train_iter, val_iter, test_iter, num_epochs, lr, try_gpu()) #2 min
    train_loss_all.append(train_loss)
    train_acc_all.append(train_acc)
    val_loss_all.append(val_loss)
    val_acc_all.append(val_acc)
    test_loss_all.append(test_loss)
    test_acc_all.append(test_acc)


buffer_train_val = np.array([train_loss_all, train_acc_all, val_loss_all, val_acc_all])
buffer_test = np.array([test_loss_all, test_acc_all])

buffer_train_val.tofile("buffer_train_val.numpy")
buffer_test.tofile("buffer_test.numpy")

## Data interpretation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

batch_size, lr, num_epochs = 256, 0.9, 10
rate_errors = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
buffer_test = np.fromfile("fashionMnist_iteration1/buffer_test.numpy")
buffer_test = buffer_test.reshape((2, len(rate_errors), 1))

test_loss_all = buffer_test[0]
test_acc_all = buffer_test[1]

plt.plot(rate_errors, test_acc_all)

In [ ]:
buffer_train_val = np.fromfile("fashionMnist_iteration1/buffer_train_val.numpy")
buffer_train_val = buffer_train_val.reshape((4, len(rate_errors), num_epochs))

train_loss_all = buffer_train_val[0]
train_acc_all = buffer_train_val[1]
val_loss_all = buffer_train_val[2]
val_acc_all = buffer_train_val[3]

best_train_acc = train_acc_all[:, -1]
best_val_acc = val_acc_all[:, -1]
error_pergentage = np.multiply(rate_errors, 100)

plt.plot(error_pergentage, best_train_acc, label='Train acc - 10 epochs')
plt.plot(error_pergentage, best_val_acc, label='Validation acc - 10 epochs')
plt.plot(error_pergentage, test_acc_all, label='Test acc - 10 epochs')
plt.xlabel('Errors in the dataset [%]')
plt.ylabel('Accuracy [%]')

plt.legend(), plt.show()

In [ ]:
print(test_acc_all)